In [1]:
!pip install tinygrad numpy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
!python3 -m tinygrad.device

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'tinygrad.device' found in sys.modules after import of package 'tinygrad', but prior to execution of 'tinygrad.device'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
  METAL     : FAIL /usr/lib/libobjc.dylib: cannot open shared object file: No such file or directory
  AMD       : FAIL No interface for AMD:0 is available
  NV        : FAIL rm_control returned 58: Invalid structure parameter
* CUDA      : PASS
                 + CUDACompiler (default)
                 + PTXCompiler (CUDA_PTX=1 to make default)
                 + NVCCCompiler (CUDA_NVCC=1 to make default)
  QCOM      : FAIL [Errno 2] No such file or directory: '/dev/kgsl-3d0'
  CL        : FAIL OpenCL Error -1001: Unknown error
  CPU       : PASS
                 + ClangJITCompiler (default)
                 + CPULLVMCompiler (CPU_LLVM=1 to make default)
                 - LVPCompiler: libtinymesa not found (MESA_PATH=/usr/lib). See https://github.

In [3]:
%env DEBUG=2

env: DEBUG=2


In [8]:
from typing import Callable
from tinygrad import Tensor, TinyJit, nn
import numpy as np
import timeit

class Model:
  def __init__(self):
    self.layers: list[Callable[[Tensor], Tensor]] = [
      nn.Conv2d(3, 32, 5), Tensor.relu,
      nn.Conv2d(32, 32, 5), Tensor.relu,
      nn.BatchNorm2d(32), Tensor.max_pool2d,
      nn.Conv2d(32, 64, 3), Tensor.relu,
      nn.Conv2d(64, 64, 3), Tensor.relu,
      nn.BatchNorm2d(64), Tensor.max_pool2d,
      lambda x: x.mean(axis=(2,3)), nn.Linear(64, 1)]

  def __call__(self, x:Tensor) -> Tensor: return x.sequential(self.layers)

if __name__ == "__main__":
  # X = np.random.randn(16, 3, 512, 512).astype(np.float32)
  # Y = np.random.randn(16, 1).astype(np.float32)
  X, Y = np.load("data/X.npy"), np.load("data/Y.npy")
  permutation = [0, 1, 5, 14, 13, 11, 8, 9, 2, 15, 4, 7, 10, 12, 3, 6]
  X_train, X_val = X[permutation[:10]], X[permutation[10:13]]
  Y_train, Y_val = Y[permutation[:10]], Y[permutation[10:13]]
  X_train, Y_train, X_val, Y_val = map(Tensor, [X_train, Y_train, X_val, Y_val])

  model = Model()
  opt = nn.optim.Adam(nn.state.get_parameters(model))
  
  @TinyJit
  def get_val_acc() -> Tensor: return ((model(X_val).squeeze()-Y_val)*Y_val.reciprocal()).abs().mean()*100
  print("starting")
  print(timeit.repeat(get_val_acc, repeat=1, number=1))
  print("done")



starting
scheduled 26 kernels in 55.32 ms
*** CUDA      81 copy    2.36M,    CUDA <- NPY                  arg  2 mem   0.06 GB tm   8494.44us/122528.94ms (      0 GFLOPS    0|0      GB/s) ['conv2d']
*** CUDA      82 En7                                            arg  1 mem   0.06 GB tm     66.94us/122529.00ms (      0 GFLOPS    0|0      GB/s) ['uniform']
*** CUDA      83 copy       24,    CUDA <- NPY                  arg  2 mem   0.06 GB tm     34.12us/122529.04ms (      0 GFLOPS    0|0      GB/s) 
*** CUDA      84 E_25_32_3                                      arg  3 mem   0.06 GB tm      8.54us/122529.05ms (     74 GFLOPS    1|2      GB/s) ['uniform']
*** CUDA      85 E                                              arg  2 mem   0.06 GB tm      7.23us/122529.05ms (      0 GFLOPS    0|0      GB/s) ['uniform']
*** CUDA      86 E_8_4                                          arg  3 mem   0.06 GB tm      6.05us/122529.06ms (      1 GFLOPS    0|0      GB/s) ['uniform']
*** CUDA      87 En1  